# BRFSS Conversion and Cleaning

In [1]:
import pandas as pd
df = pd.read_sas(r"C:\Users\bruce\OneDrive\Desktop\project_part_ii\data_lake\raw\LLCP2024.XPT")


In [2]:
df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


C:\Users\bruce\AppData\Local\Temp\ipykernel_5604\1659967169.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)


In [3]:
df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_LCSCTSN,_LCSPSTF,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK3,_RFDRHV9,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,2.0,02282024,02,28,2024,1100.0,2024000001,2.024000e+09,1.0,...,NaN,9.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,2.0,2.0
1,1.0,2.0,02212024,02,21,2024,1100.0,2024000002,2.024000e+09,1.0,...,4.0,9.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0
2,1.0,2.0,02212024,02,21,2024,1100.0,2024000003,2.024000e+09,1.0,...,4.0,2.0,1.0,1.000000e+02,2.0,1.400000e+03,1.0,NaN,NaN,2.0
3,1.0,2.0,02282024,02,28,2024,1100.0,2024000004,2.024000e+09,1.0,...,NaN,9.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0
4,1.0,2.0,02212024,02,21,2024,1100.0,2024000005,2.024000e+09,1.0,...,3.0,9.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457665,78.0,7.0,07122024,07,12,2024,1200.0,2024001344,2.024001e+09,NaN,...,NaN,NaN,9.0,9.000000e+02,9.0,9.990000e+04,9.0,9.0,9.0,NaN
457666,78.0,7.0,07182024,07,18,2024,1100.0,2024001345,2.024001e+09,NaN,...,4.0,2.0,1.0,1.000000e+02,9.0,3.500000e+03,2.0,2.0,2.0,1.0
457667,78.0,7.0,07172024,07,17,2024,1100.0,2024001346,2.024001e+09,NaN,...,4.0,9.0,1.0,7.000000e+00,1.0,1.400000e+02,1.0,1.0,1.0,2.0
457668,78.0,7.0,07202024,07,20,2024,1200.0,2024001347,2.024001e+09,NaN,...,NaN,NaN,9.0,9.000000e+02,9.0,9.990000e+04,9.0,NaN,NaN,NaN


In [15]:
keep = ['_STATE', '_AGE80', 'SEXVAR', '_EDUCAG', 'INCOME3', 
         '_SMOKER3', 'DRNKANY6', 'EXERANY2', '_RFDRHV9', '_BMI5',
         'DIABETE4', 'CVDCRHD4', 'CVDSTRK3',
         'FLUSHOT7', '_PNEUMO3', 'SEQNO', '_PSU']

df_sub = df[keep].copy()


In [16]:
rename_dict = {
    '_STATE': 'state_fips',
    '_AGE80': 'age',
    'SEXVAR': 'gender',
    '_EDUCAG': 'education_level',
    'INCOME3': 'income_level',
    '_SMOKER3': 'smoking_status',
    'DRNKANY6': 'alcohol_use',
    'EXERANY2': 'exercise_status',
    '_RFDRHV9': 'heavy_drinker',
    '_BMI5': 'bmi',
    'DIABETE4': 'diabetes_status',
    'CVDCRHD4': 'heart_disease',
    'CVDSTRK3': 'stroke_history',
    'FLUSHOT7': 'flu_shot',
    '_PNEUMO3': 'pneumonia_shot',
    'SEQNO': 'sequence_id',
    '_PSU': 'primary_sampling_unit'
}

df_sub.rename(columns=rename_dict, inplace=True)


In [17]:
import numpy as np

def map_binary(x):
    if x in [1.0, 1]: return "Yes"
    elif x in [2.0, 2]: return "No"
    elif x in [7.0, 9.0, 77.0, 99.0, 777.0, 999.0, np.nan]: return np.nan
    else: return np.nan

gender_map = {1.0: "Male", 2.0: "Female"}

education_map = {
    1.0: "Did not graduate high school",
    2.0: "Graduated high school",
    3.0: "Attended College or Technical School",
    4.0: "Graduated from College or Technical School",
    9.0: np.nan
}

state_map = {
    1:  "Alabama",
    2:  "Alaska",
    4:  "Arizona",
    5:  "Arkansas",
    6:  "California",
    8:  "Colorado",
    9:  "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming",
    66: "Guam",
    72: "Puerto Rico",
    78: "Virgin Islands"
}


income_map = {
    1: "Less than $15,000",
    2: "$15,000–$25,000",
    3.0: "$25,000–$35,000",
    4.0: "$35,000–$50,000",
    5.0: "$50,000–$100,000",
    6.0: "$100,000–$200,000",
    7.0: "$200,000 or more",
    9.0: np.nan
}

smoker_map = {
    1.0: "Smokes every day",
    2.0: "Smokes some days",
    3.0: "Former smoker",
    4.0: "Never smoked",
    9.0: np.nan
}

diabete_map = {
    1.0: "Yes",
    2.0: "Yes, but only told during pregnancy",
    3.0: "No",
    4.0: "No, but pre-diabetes or borderline diabetes",
    7.0: np.nan,
    9.0: np.nan
}

heavy_drinker_map = diabete_map = {
    1.0: "No",
    2.0: "Yes",
    9.0: np.nan
}

binary_cols = [
     'alcohol_use', 'exercise_status',
      'heart_disease',
    'stroke_history', 'flu_shot', 'pneumonia_shot'
]

for col in binary_cols:
    df_sub[col] = df_sub[col].apply(map_binary)

df_sub['gender'] = df_sub['gender'].map(gender_map)
df_sub['education_level'] = df_sub['education_level'].map(education_map)
df_sub['income_level'] = df_sub['income_level'].map(income_map)
df_sub['smoking_status'] = df_sub['smoking_status'].map(smoker_map)
df_sub['diabetes_status'] = df_sub['diabetes_status'].map(diabete_map)
df_sub['state_name'] = df_sub['state_fips'].map(state_map)


df_sub['bmi'] = df_sub['bmi'] / 100.0

print("Missing value count per column:")
print(df_sub.isna().sum())

print(df_sub.head())

df_sub.to_csv(
    r"C:\Users\bruce\OneDrive\Desktop\project_part_ii\data_lake\processed\brfss_clean_semantic.csv",
    index=False
)


Missing value count per column:
state_fips                    0
age                           0
gender                        0
education_level            2363
income_level             251270
smoking_status            32022
alcohol_use               43777
exercise_status            1315
heavy_drinker                 0
bmi                       43037
diabetes_status          388466
heart_disease              4578
stroke_history             1452
flu_shot                  44366
pneumonia_shot           305865
sequence_id                   0
primary_sampling_unit         0
state_name                    0
dtype: int64
   state_fips   age  gender                             education_level  \
0         1.0  78.0  Female                       Graduated high school   
1         1.0  80.0    Male  Graduated from College or Technical School   
2         1.0  59.0    Male        Attended College or Technical School   
3         1.0  80.0    Male  Graduated from College or Technical School   
4   

# CDC WONDER Conversion and Cleaning


In [30]:
!pip install html5lib

In [37]:

# -----------------------------
# 1. File paths
# -----------------------------
input_path = r"C:\Users\bruce\OneDrive\Desktop\project_part_ii\data_lake\raw\cdc_wonder_raw.csv"
output_path = r"C:\Users\bruce\OneDrive\Desktop\project_part_ii\data_lake\processed\cdc_wonder_clean.csv"

# -----------------------------
# 2. Import Excel (first sheet)
# -----------------------------
df = pd.read_csv(input_path, skiprows=0)

# -----------------------------
# 3. Remove unnecessary columns
# -----------------------------
cols_to_drop = [
    "Notes",
    "Sex Code",
    "Population",          # contains "Not Applicable"
    "Crude Rate",          # same issue
]

# remove duplicates of age-group columns if present
duplicate_age_cols = [col for col in df.columns if "Ten-Year Age Grou" in col and col != "Ten-Year Age Groups"]

df.drop(columns=[c for c in cols_to_drop if c in df.columns], inplace=True)
df.drop(columns=duplicate_age_cols, inplace=True)

# -----------------------------
# 4. Rename columns → match BRFSS schema style
# -----------------------------
rename_dict = {
    "Residence State": "state_name",
    "Residence State Code": "state_fips",
    "Sex": "sex",
    "Ten-Year Age Groups": "age_group",
    "Deaths": "deaths"
}

df.rename(columns=rename_dict, inplace=True)



In [39]:
# -----------------------------
# 5. Drop "Not Applicable" & clean numeric columns
# -----------------------------
df["deaths"] = pd.to_numeric(df["deaths"], errors="coerce")


# -----------------------------
# 7. Reorder / keep useful columns only
# -----------------------------
df = df[["state_fips", "state_name", "sex", "age_group", "deaths"]]

# -----------------------------
# 8. Save cleaned version
# -----------------------------
df.to_csv(output_path, index=False)

print("CDC WONDER cleaned dataset saved to:")
print(output_path)
print(df.head())


CDC WONDER cleaned dataset saved to:
C:\Users\bruce\OneDrive\Desktop\project_part_ii\data_lake\processed\cdc_wonder_clean.csv
   state_fips state_name   sex    age_group  deaths
0         1.0    Alabama  Male     < 1 year    62.0
1         1.0    Alabama  Male    1-4 years    14.0
2         1.0    Alabama  Male  15-24 years    69.0
3         1.0    Alabama  Male  25-34 years   127.0
4         1.0    Alabama  Male  35-44 years   254.0


In [38]:
df

,state_name,state_fips,sex,age_group,deaths
0,Alabama,1.0,Male,< 1 year,62.0
1,Alabama,1.0,Male,1-4 years,14.0
2,Alabama,1.0,Male,15-24 years,69.0
3,Alabama,1.0,Male,25-34 years,127.0
4,Alabama,1.0,Male,35-44 years,254.0
...,...,...,...,...,...
431,NaN,NaN,NaN,NaN,NaN
432,NaN,NaN,NaN,NaN,NaN
433,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,NaN
